In [39]:
from bs4 import BeautifulSoup 
import requests 
import requests.exceptions 
from urllib.parse import urlsplit
from urllib.request import Request
from urllib.request import urlopen
from flask import request
from collections import deque 
import re 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


import os
import glob
import time
import datetime

import pandas as pd
import numpy as np

import sys

In [40]:
def init_driver():
    
    global driver
    #driver = webdriver.Chrome()

    driver = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(driver, 7)

    driver.get('https://www.verif.com/')

In [41]:
def get_info(raisons,cas,formes,capitaux,codes,gerants,descriptions):
    for ii in range(1,25):
        try:
            line = driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[1]'.format(ii)).text.lower()
            
            ## RAISON SOCIALE
            if "raison sociale" in line:
                raisons.append(driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[2]'.format(ii)).text)
            
            ## CA
            if "chiffre d'affaire" in line:
                cas.append(driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[2]'.format(ii)).text.split('\n')[0])
            
            ## FORME JURIDIQUE
            if "forme juridique" in line:
                formes.append(driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[2]'.format(ii)).text.split('\n')[0])
            
            ## CAPITAL SOCIAL
            if "capital social" in line:
                capitaux.append(driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[2]'.format(ii)).text.split('\n')[0])
            
            ## APE / NAF
            if 'ape' in line or 'naf' in line:
                codes.append(driver.find_element_by_xpath('//*[@class="table infoGen hidden-smallDevice"]/tbody/tr[{}]/td[2]'.format(ii)).text.split('\n')[0])
                           
        except:
            pass
    
    ## GERANT(S)
    try:
        gerants.append(driver.find_element_by_xpath('//*[@class="table table-default dirigeants"]').text.replace('\n',' - '))
    except:
        gerants.append('error')
        
    ## DESCRIPTION
    try:
        descriptions.append(driver.find_element_by_xpath('//*[@class="accroche"]').text.replace('\n',' ; '))
    except:
        descriptions.append('error')
    
    for arr in [raisons,cas,formes,capitaux,codes,gerants,descriptions] :
        if len(arr) < len(max( [raisons,cas,formes,capitaux,codes,gerants,descriptions] , key = lambda x : len(x))):
            arr.append('error')
    
    return raisons,cas,formes,capitaux,codes,gerants,descriptions   

In [42]:
from bs4 import BeautifulSoup

import requests

import requests.exceptions

from urllib.parse import urlsplit

from urllib.request import Request

from urllib.request import urlopen

from flask import request

from collections import deque

import re

import http.cookiejar
import urllib.request
from urllib.request import urlopen
import pandas as pd
from pyxlsb import open_workbook as open_xlsb
 

# a queue of urls to be crawled

#new_urls = deque(['https://www.ubaldi.com/accueil/'])
df = []
df_output=pd.DataFrame(columns=['Website','Siren','Raison sociale structure','cas','codes','capitaux','formes','gerants','Elements number'])

with open_xlsb('test.csv') as wb:
    with wb.get_sheet(1) as sheet:
        for row in sheet.rows():
            df.append([item.v for item in row])

df = pd.DataFrame(df[1:], columns=df[0])

list_URL=df['Website']
integer=0
for i in list_URL:

    URL=(str("https://www.")+i)
    #URL='https://www.protextyl.com/'
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}
    # a set of urls that we have already crawled

    processed_urls = set()



    # a set of crawled emails

    SIRET = []
    SIREN= []


    def TestAccessible(the_uri):

    # l'URL du site web est valide, on va chercher s'il existe un lien vers la page contact et retourner ce lien

        #html_page = urlopen(the_uri)
        req = urllib.request.Request(the_uri, headers=hdr)
        opener = urllib.request.build_opener()
        response = opener.open(req)
        try:
            html_page = urllib.request.urlopen(req)
        except urllib.HTTPError as e:
            print(e.fp.read())
        #response is now a string you can search through containing the page's html

        soup = BeautifulSoup(html_page, 'html.parser')

        LinkList = []

        for Link in soup.find_all('a'):



            LinkFound=Link.get('href')

            if LinkFound is not None:

                if "http" in LinkFound:

                    LinkList.append(str(LinkFound))

                else:

                    LinkList.append(str(the_uri+LinkFound))



        contactList = list(filter(lambda x: "mentions-legales" in x,LinkList))





        return (contactList)



    new_urls=TestAccessible(URL)

    print(new_urls)

    print(len(new_urls))

    i=0



    # process urls one by one until we exhaust the queue

    while i<len(new_urls):

        new_url=deque([new_urls[i]])

        # move next url from the queue to the set of processed urls

        url = new_url.popleft()

        processed_urls.add(url)



        # extract base url to resolve relative links

        parts = urlsplit(url)

        base_url = "{0.scheme}://{0.netloc}".format(parts)

        path = url[:url.rfind('/')+1] if '/' in parts.path else url



        # get url's content

        print("Processing %s" % url)

        try:

            response = requests.get(url)

            i+=1

        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):

            i+=1

            # ignore pages with errors

            continue



        requete = requests.get(url)

        page = requete.content

        soup = BeautifulSoup(page)

        #SIRET_find = set(re.findall(r'[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]+\s[0-9][0-9][0-9][0-9][0-9]', soup.text, re.I))
        #if len(SIRET_find)!=0:
            #for sir in SIRET_find:
                #SIRET.append(sir) 
        
        #SIRET_find = set(re.findall(r'[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9][0-9][0-9][0-9]', soup.text, re.I))
        #if len(SIRET_find)!=0:
            #for sir in SIRET_find:
                #SIRET.append(sir) 

        #SIRET_find = set(re.findall(r'[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', soup.text, re.I))
        #if len(SIRET_find)!=0:
            #for sir in SIRET_find:
                #SIRET.append(sir) 

        #SIRET_find = set(re.findall(r'^[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9]', soup.text, re.I))
        #if len(SIRET_find)!=0:
            #for sir in SIRET_find:
                #SIRET.append(sir) 



        SIREN_find = set(re.findall(r'[0-9][0-9][0-9]+\s[0-9][0-9][0-9]+\s[0-9][0-9][0-9]', soup.text, re.I)) 
        if len(SIREN_find)!=0:
            for sir in SIREN_find:
                sir=sir.replace(" ","")
                SIREN.append(sir) 
            
        SIREN_find = set(re.findall(r'[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', soup.text, re.I)) 
        if len(SIREN_find)!=0:
            for sir in SIREN_find:
                SIREN.append(sir) 


    #taille=max(len(SIREN),len(SIRET))
    taille=len(SIREN)
    
    while len(SIREN)!=taille:
        SIREN.extend([''])
        
    #while len(SIRET)!=taille: 
        #SIRET.extend([''])
        
    for j in range(0,taille):
       
        df_output.loc[integer,"Website"]=URL
        df_output.loc[integer,"Siren"]=SIREN[j]
        #df_output.loc[integer,"Siret"]=SIRET[j]
        
        integer+=1
        
    

    print(SIREN)

    
print(df_output)

['https://www.vetementpro.com/content/2-mentions-legales']
1
Processing https://www.vetementpro.com/content/2-mentions-legales
['539871657', '721525133', '104734598', '160510667']
['https://www.protextyl.com/content/8-mentions-legales']
1
Processing https://www.protextyl.com/content/8-mentions-legales
['13807006360', '531801587', '719210149', '159076348', '160510667']
['https://www.oxwork.com/mentions-legales']
1
Processing https://www.oxwork.com/mentions-legales
['531173615', '160458228', '075311736', '160458424', '864000000']
['https://www.protecthoms.com/mentions-legales/']
1
Processing https://www.protecthoms.com/mentions-legales/
['390145563', '723901455', '012345678', '504731036', '303325320']
                        Website        Siren Raison sociale structure  cas  \
0   https://www.vetementpro.com    539871657                      NaN  NaN   
1   https://www.vetementpro.com    721525133                      NaN  NaN   
2   https://www.vetementpro.com    104734598             

In [43]:
from IPython.display import clear_output

init_driver()

raisons = []
cas = []
formes = []
capitaux = []
codes = []
gerants = []
descriptions = []
start = time.time()
nan_cols=[]
for i,siren in enumerate(df_output["Siren"]):
    
    try:
        driver.get('https://www.verif.com/societe/' + str(siren))
    except:
        try:
            driver.quit()
            init_driver()
            driver.get('https://www.verif.com/societe/' + str(siren))
        except:
            next(sirens)
            driver.get('https://www.verif.com/societe/' + str(siren))
                  
    get_info(raisons,cas,formes,capitaux,codes,gerants,descriptions)
        
    step = time.time()
    avgtime = (step - start)/len(raisons)
                    
    clear_output()
    print(len(raisons) , ' - - ' , 'average time : {0:.2f}'.format(avgtime), ' - - ' , raisons[-1] , ' - - ' , cas[-1] , ' - - ' , formes[-1] , ' - - ' , capitaux[-1] , ' - - ' , codes[-1] , ' - - ' , gerants[-1])
    df_output['Raison sociale structure'][i] = raisons[-1]
    df_output['cas'][i] = cas[-1]
    df_output['codes'][i] =  codes[-1]
    df_output['capitaux'][i] = capitaux[-1]
    df_output['formes'][i] =  formes[-1] 
    df_output['gerants'][i] =  gerants[-1] 
    col_output=['Raison sociale structure','cas','codes','capitaux','formes','gerants']
    
    df_output['Elements number'][i]=len([col for col in col_output if df_output[col][i]!="error"])

    
df_output['Elements number'] = df_output['Elements number'].replace(0, np.nan)
df_output = df_output.dropna(axis=0, subset=['Elements number'])

print(df_output)


if os.path.exists("output.xlsx"):
    os.remove("output.xlsx")
else:
    print("The file does not exist")

writer_2 = pd.ExcelWriter('output.xlsx')
# write dataframe to excel
df_output.to_excel(writer_2)
# save the excel
writer_2.save()



19  - -  average time : 4.57  - -  error  - -  error  - -  error  - -  error  - -  error  - -  error
                        Website      Siren Raison sociale structure  \
0   https://www.vetementpro.com  539871657               COMMERSENS   
5     https://www.protextyl.com  531801587                   MEYDES   
9        https://www.oxwork.com  531173615                  PROSHOP   
14  https://www.protecthoms.com  390145563             PROTECT'HOMS   

             cas                                              codes  \
0    4 925 311 €     4791A / Vente à distance sur catalogue général   
5          error  4642Z / Commerce de gros (commerce interentrep...   
9      130 118 €  4791B / Vente à distance sur catalogue spécialisé   
14  21 502 633 €  4649Z / Commerce de gros (commerce interentrep...   

       capitaux                          formes  \
0      81 000 €                            SARL   
5      45 000 €                            SARL   
9       2 000 €  Société par actio